In [1]:
%cd /aiffel

from copy import deepcopy
import json

import torch
from torch.optim import Adam
from chatgpt.models.base import RewardModel
from chatgpt.models.gpt import GPTActor, GPTCritic
from chatgpt.trainer import PPOTrainer
from chatgpt.trainer.strategies import NaiveStrategy
from transformers import AutoTokenizer

/aiffel


In [2]:
with NaiveStrategy().model_init_context():
    actor = GPTActor(pretrained='aiffel/KoChatGPT/output_2_SFT', lora_rank=0).to(torch.cuda.current_device())
    critic = GPTCritic(pretrained='aiffel/KoChatGPT/output_2_RM', lora_rank=0).to(torch.cuda.current_device())

    tokenizer = AutoTokenizer.from_pretrained(
        'EleutherAI/polyglot-ko-1.3b', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right", 
        model_max_length=512
    )

    initial_model = deepcopy(actor)
    reward_model = RewardModel(deepcopy(critic.model), deepcopy(critic.value_head)).to(torch.cuda.current_device())

You are using a model of type gpt_neox to instantiate a model of type gpt2. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at aiffel/KoChatGPT/output_2_SFT were not used when initializing GPT2LMHeadModel: ['gpt_neox.layers.11.input_layernorm.weight', 'gpt_neox.layers.1.post_attention_layernorm.bias', 'gpt_neox.layers.4.mlp.dense_h_to_4h.weight', 'gpt_neox.layers.21.mlp.dense_h_to_4h.bias', 'gpt_neox.layers.16.mlp.dense_h_to_4h.weight', 'gpt_neox.layers.10.attention.dense.bias', 'gpt_neox.layers.4.attention.dense.bias', 'gpt_neox.layers.13.attention.query_key_value.weight', 'gpt_neox.layers.14.mlp.dense_h_to_4h.weight', 'gpt_neox.layers.21.attention.dense.weight', 'gpt_neox.layers.8.mlp.dense_4h_to_h.weight', 'gpt_neox.layers.11.attention.masked_bias', 'gpt_neox.layers.20.post_attention_layernorm.weight', 'gpt_neox.layers.22.attention.bias', 'gpt_neox.layers.5.attention.rotary_emb.inv_freq', 'gpt_neox.layers.6.post_atten

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
actor_optim = Adam(actor.parameters(), lr=5e-6)
critic_optim = Adam(critic.parameters(), lr=5e-6)

In [4]:
(actor, actor_optim), (critic, critic_optim), reward_model, initial_model = NaiveStrategy().prepare(
    (actor, actor_optim), (critic, critic_optim), reward_model, initial_model)

In [5]:
with open('aiffel/KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)
    list_prompt = [tmp['prompt'] for tmp in list_data_dict]

def tokenize_fn(texts):
    batch = tokenizer(texts, return_tensors='pt', max_length=96, padding=True, truncation=True)
    return {k: v.cuda() for k, v in batch.items()}

In [6]:
print(tokenize_fn('It takes something more than intelligence to act intelligently.'))

{'input_ids': tensor([[   44,    87,  3628, 12504,  5862,  9244,  6723,  5145,    75,  4541,
          7789, 10101, 22253,  2818, 10903,    87, 15075,  7135, 18190, 13384,
           224, 16683, 10903,    87, 15075,  7135,  6522, 16736,    17]],
       device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1]], device='cuda:0')}


In [7]:
len(list_prompt)

12000

In [8]:
trainer = PPOTrainer(NaiveStrategy(),
                     actor,
                     critic,
                     reward_model,
                     initial_model,
                     actor_optim,
                     critic_optim,
                     max_epochs=1,  
                     train_batch_size=2, 
                     tokenizer=tokenize_fn,
                     max_length=64,
                     do_sample=True,
                     temperature=1.0,
                     top_k=50,
                     pad_token_id=tokenizer.pad_token_id,
                     eos_token_id=tokenizer.eos_token_id)

In [9]:
trainer.fit(list_prompt, 
            num_episodes=10,  
            max_timesteps=3,
            update_timesteps=3)



Episode [1/10]:   0%|          | 0/1 [00:22<?, ?it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 trainer.fit(list_prompt,                                                                     │
│   2 │   │   │   num_episodes=10,                                                                 │
│   3 │   │   │   max_timesteps=1,                                                                 │
│   4 │   │   │   update_timesteps=1)                                                              │
│                                                                                                  │
│ /aiffel/chatgpt/trainer/base.py:118 in fit                                                       │
│                                                                                                  │
│   115 │   │   │   │   self._on_make_experience_end(experience)                                   │
│   116 │   │   │   │   self.replay_buffer.append(experience)                                      │
│   117 │   │   │   │   if time % update_timesteps == 0:                                           │
│ ❱ 118 │   │   │   │   │   self._learn()                                                          │
│   119 │   │   │   │   │   self.replay_buffer.clear()                                             │
│   120 │   │   │   self._on_episode_end(episode)                                                  │
│   121 │   │   self._on_fit_end()                                                                 │
│                                                                                                  │
│ /aiffel/chatgpt/trainer/base.py:93 in _learn                                                     │
│                                                                                                  │
│    90 │   │   │   │   for experience in pbar:                                                    │
│    91 │   │   │   │   │   self._on_learn_batch_start()                                           │
│    92 │   │   │   │   │   experience.to_device(device)                                           │
│ ❱  93 │   │   │   │   │   metrics = self.training_step(experience)                               │
│    94 │   │   │   │   │   self._on_learn_batch_end(metrics, experience)                          │
│    95 │   │   │   │   │   pbar.set_postfix(metrics)                                              │
│    96 │   │   │   │   self._on_learn_epoch_end(epoch)                                            │
│                                                                                                  │
│ /aiffel/chatgpt/trainer/ppo.py:88 in training_step                                               │
│                                                                                                  │
│    85 │   │   │   │   │   │   │   │   │   │   experience.action_log_probs,                       │
│    86 │   │   │   │   │   │   │   │   │   │   experience.advantages,                             │
│    87 │   │   │   │   │   │   │   │   │   │   action_mask=experience.action_mask)                │
│ ❱  88 │   │   self.strategy.backward(actor_loss, self.actor, self.actor_optim)                   │
│    89 │   │   self.strategy.optimizer_step(self.actor_optim)                                     │
│    90 │   │   self.actor_optim.zero_grad()                                                       │
│    91                                                                                            │
│                                                                                                  │
│ /aiffel/chatgpt/trainer/strategies/naive.py:19 in backward                                       │
│                                                            

In [ ]:
actor.model.save_pretrained('aiffel/KoChatGPT/output_3_PPO_2')

In [ ]:
def generation(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    outputs = actor.generate(input_ids,
                             max_length=250,
                             do_sample=True,
                             top_k=50,
                             top_p=0.95,
                             num_return_sequences=1)
    output = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)[0]
    print()
    print(output)
    return output

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = [
    '불고기용 고기 한우에요?', 
    '리처드 닉슨이 43대 부통령직을 수행한 년도는?', 
    '시카고 오헤어 국제공항은 어디에 있어',
    '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt': tmp}) for tmp in list_prompt]

for input_text in list_prompt:
    output = generation(input_text)